# WM Statefarm Notebook


## Relevant sources:

https://www.kaggle.com/c/state-farm-distracted-driver-detection

http://ec2-35-167-61-73.us-west-2.compute.amazonaws.com:8888/notebooks/dogs_cats_redux.ipynb#Dogs-vs-Cat-Redux

http://ec2-35-167-61-73.us-west-2.compute.amazonaws.com:8888/edit/utils.py

http://ec2-35-167-61-73.us-west-2.compute.amazonaws.com:8888/edit/vgg16.py

http://wiki.fast.ai/index.php/Lesson_2_Notes

## Basic setup

In [1]:
import os, sys

%pwd

u'/home/ubuntu/nbs'

In [2]:
current_dir = os.getcwd()
LESSON_HOME_DIR = current_dir
DATA_HOME_DIR = LESSON_HOME_DIR + '/data/statefarm'

In [3]:
from utils import *
from vgg16 import Vgg16

# instantiate plotting tool
%matplotlib inline

Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)
/home/ubuntu/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)
Using Theano backend.


## Download & structure data

In [6]:
# data was downloaded via command line
# !kg download -c state-farm-distracted-driver-detection

In [12]:
%cd $DATA_HOME_DIR

%mkdir results
%mkdir valid

%mkdir -p test/unknown

%mkdir -p sample/results
%mkdir -p sample/train
%mkdir -p sample/test
%mkdir -p sample/valid

/home/ubuntu/nbs/data/statefarm


In [13]:
# sort train data into proper classes
# train data already sorted into classes
# [ ]

In [34]:
%cd $DATA_HOME_DIR/test

%mv *.jpg unknown

/home/ubuntu/nbs/data/statefarm/test


In [72]:
# copy /train folders data structure for replication in valid/, sample/train, and sample/valid
%cd $DATA_HOME_DIR/train

classes = os.listdir('.')

/home/ubuntu/nbs/data/statefarm/train
/home/ubuntu/nbs/data/statefarm
/home/ubuntu/nbs/data/statefarm/valid


In [46]:
# re-create folder structure in /train

%cd $DATA_HOME_DIR/valid
for x in classes:
    %mkdir $x
    
%cd $DATA_HOME_DIR/sample/train
for x in classes:
    %mkdir $x

%cd $DATA_HOME_DIR/sample/valid
for x in classes:
    %mkdir $x

/home/ubuntu/nbs/data/statefarm/valid
mkdir: cannot create directory ‘c7’: File exists
mkdir: cannot create directory ‘c8’: File exists
mkdir: cannot create directory ‘c6’: File exists
mkdir: cannot create directory ‘c0’: File exists
mkdir: cannot create directory ‘c2’: File exists
mkdir: cannot create directory ‘c4’: File exists
mkdir: cannot create directory ‘c9’: File exists
mkdir: cannot create directory ‘c1’: File exists
mkdir: cannot create directory ‘c3’: File exists
mkdir: cannot create directory ‘c5’: File exists
/home/ubuntu/nbs/data/statefarm/sample/train
/home/ubuntu/nbs/data/statefarm/sample/valid


In [70]:
# move / copy appropriate sets of training data into valid
%cd $DATA_HOME_DIR/train

for x in classes:
    %cd $DATA_HOME_DIR/train/$x
    g = glob('*.jpg')
    shuf = np.random.permutation(g)
    for i in range(200): os.rename(shuf[i], DATA_HOME_DIR+'/valid/'+x+'/'+shuf[i])

/home/ubuntu/nbs/data/statefarm/train
/home/ubuntu/nbs/data/statefarm/train/c7
/home/ubuntu/nbs/data/statefarm/train/c8
/home/ubuntu/nbs/data/statefarm/train/c6
/home/ubuntu/nbs/data/statefarm/train/c0
/home/ubuntu/nbs/data/statefarm/train/c2
/home/ubuntu/nbs/data/statefarm/train/c4
/home/ubuntu/nbs/data/statefarm/train/c9
/home/ubuntu/nbs/data/statefarm/train/c1
/home/ubuntu/nbs/data/statefarm/train/c3
/home/ubuntu/nbs/data/statefarm/train/c5


In [71]:
# move / copy appropriate sets of training data into sample
from shutil import copyfile

%cd $DATA_HOME_DIR/train
for x in classes:
    %cd $DATA_HOME_DIR/train/$x
    g = glob('*.jpg')
    shuf = np.random.permutation(g)
    for i in range(20): copyfile(shuf[i], DATA_HOME_DIR+'/sample/train/'+x+'/'+shuf[i])

%cd $DATA_HOME_DIR/valid
for x in classes:
    %cd $DATA_HOME_DIR/valid/$x
    g = glob('*.jpg')
    shuf = np.random.permutation(g)
    for i in range(5): copyfile(shuf[i], DATA_HOME_DIR+'/sample/valid/'+x+'/'+shuf[i])

/home/ubuntu/nbs/data/statefarm/train
/home/ubuntu/nbs/data/statefarm/train/c7
/home/ubuntu/nbs/data/statefarm/train/c8
/home/ubuntu/nbs/data/statefarm/train/c6
/home/ubuntu/nbs/data/statefarm/train/c0
/home/ubuntu/nbs/data/statefarm/train/c2
/home/ubuntu/nbs/data/statefarm/train/c4
/home/ubuntu/nbs/data/statefarm/train/c9
/home/ubuntu/nbs/data/statefarm/train/c1
/home/ubuntu/nbs/data/statefarm/train/c3
/home/ubuntu/nbs/data/statefarm/train/c5
/home/ubuntu/nbs/data/statefarm/valid
/home/ubuntu/nbs/data/statefarm/valid/c7
/home/ubuntu/nbs/data/statefarm/valid/c8
/home/ubuntu/nbs/data/statefarm/valid/c6
/home/ubuntu/nbs/data/statefarm/valid/c0
/home/ubuntu/nbs/data/statefarm/valid/c2
/home/ubuntu/nbs/data/statefarm/valid/c4
/home/ubuntu/nbs/data/statefarm/valid/c9
/home/ubuntu/nbs/data/statefarm/valid/c1
/home/ubuntu/nbs/data/statefarm/valid/c3
/home/ubuntu/nbs/data/statefarm/valid/c5


In [73]:
%cd $DATA_HOME_DIR

with open('driver_imgs_list.csv') as f:
    driver_list = f.read().splitlines()

driver_list = driver_list[1:]
print driver_list[:3]
print set([x[:4] for x in driver_list])
first_driver = [x for x in driver_list if x[:4] == "p002"]
second_driver = [x for x in driver_list if x[:4] == "p049"]
third_driver = [x for x in driver_list if x[:4] == "p022"]

for x in first_driver:
    tr_path = DATA_HOME_DIR+'/train/'+x[5:7]+'/'+x[8:]
    val_path = DATA_HOME_DIR+'/valid/'+x[5:7]+'/'+x[8:]
    os.rename(tr_path, val_path)
    
for x in second_driver:
    tr_path = DATA_HOME_DIR+'/train/'+x[5:7]+'/'+x[8:]
    val_path = DATA_HOME_DIR+'/valid/'+x[5:7]+'/'+x[8:]
    os.rename(tr_path, val_path)
    
for x in third_driver:
    tr_path = DATA_HOME_DIR+'/train/'+x[5:7]+'/'+x[8:]
    val_path = DATA_HOME_DIR+'/valid/'+x[5:7]+'/'+x[8:]
    os.rename(tr_path, val_path)

/home/ubuntu/nbs/data/statefarm
['p002,c0,img_44733.jpg', 'p002,c0,img_72999.jpg', 'p002,c0,img_25094.jpg']
set(['p022', 'p049', 'p021', 'p026', 'p002', 'p041', 'p042', 'p045', 'p047', 'p024', 'p072', 'p075', 'p016', 'p015', 'p014', 'p039', 'p012', 'p035', 'p052', 'p051', 'p050', 'p056', 'p066', 'p064', 'p061', 'p081'])


## Build & train model

In [4]:
# set proper paths
%cd $DATA_HOME_DIR

path = DATA_HOME_DIR + '/' # 
test_path = DATA_HOME_DIR + '/test/' # use all of the test data
results_path = path + 'results/'
train_path = path + 'train/'
valid_path = path + 'valid/'

/home/ubuntu/nbs/data/statefarm


### Using ImageNet

In [5]:
# import Vgg helper class
vgg = Vgg16()

In [6]:
vgg.model.summary();

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lambda_1 (Lambda)                (None, 3, 224, 224)   0           lambda_input_1[0][0]             
____________________________________________________________________________________________________
zeropadding2d_1 (ZeroPadding2D)  (None, 3, 226, 226)   0           lambda_1[0][0]                   
____________________________________________________________________________________________________
convolution2d_1 (Convolution2D)  (None, 64, 224, 224)  1792        zeropadding2d_1[0][0]            
____________________________________________________________________________________________________
zeropadding2d_2 (ZeroPadding2D)  (None, 64, 226, 226)  0           convolution2d_1[0][0]            
___________________________________________________________________________________________

In [7]:
# set Vgg constants
batch_size = 64
num_epochs = 1

In [8]:
# get batch iterators
batches = vgg.get_batches(path=train_path,batch_size=batch_size)
val_batches = vgg.get_batches(path=valid_path,batch_size=batch_size*2)

Found 20424 images belonging to 10 classes.
Found 2000 images belonging to 10 classes.


In [9]:
# fine tune model for statefarm data
vgg.model.pop();
for layer in vgg.model.layers: layer.trainable = False
vgg.model.add(Dense(batches.nb_class,input_shape=(4096,),activation='softmax'))
vgg.model.compile(optimizer=RMSprop(),loss='categorical_crossentropy',metrics=['accuracy'])
vgg.model.summary();

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lambda_1 (Lambda)                (None, 3, 224, 224)   0           lambda_input_1[0][0]             
____________________________________________________________________________________________________
zeropadding2d_1 (ZeroPadding2D)  (None, 3, 226, 226)   0           lambda_1[0][0]                   
____________________________________________________________________________________________________
convolution2d_1 (Convolution2D)  (None, 64, 224, 224)  0           zeropadding2d_1[0][0]            
____________________________________________________________________________________________________
zeropadding2d_2 (ZeroPadding2D)  (None, 64, 226, 226)  0           convolution2d_1[0][0]            
___________________________________________________________________________________________

In [ ]:
vgg.model.load_weights(results_path+'ft4_0.h5')
num_arch = 4

In [16]:
for layer in vgg.model.layers:
    if type(layer) is Dense:
        layer.trainable = True
vgg.model.summary();

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lambda_1 (Lambda)                (None, 3, 224, 224)   0           lambda_input_1[0][0]             
____________________________________________________________________________________________________
zeropadding2d_1 (ZeroPadding2D)  (None, 3, 226, 226)   0           lambda_1[0][0]                   
____________________________________________________________________________________________________
convolution2d_1 (Convolution2D)  (None, 64, 224, 224)  0           zeropadding2d_1[0][0]            
____________________________________________________________________________________________________
zeropadding2d_2 (ZeroPadding2D)  (None, 64, 226, 226)  0           convolution2d_1[0][0]            
___________________________________________________________________________________________

In [18]:
num_arch = 4

In [25]:
for layer in vgg.model.layers:
    layer.trainable = True
vgg.model.summary();

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lambda_1 (Lambda)                (None, 3, 224, 224)   0           lambda_input_1[0][0]             
____________________________________________________________________________________________________
zeropadding2d_1 (ZeroPadding2D)  (None, 3, 226, 226)   0           lambda_1[0][0]                   
____________________________________________________________________________________________________
convolution2d_1 (Convolution2D)  (None, 64, 224, 224)  1792        zeropadding2d_1[0][0]            
____________________________________________________________________________________________________
zeropadding2d_2 (ZeroPadding2D)  (None, 64, 226, 226)  0           convolution2d_1[0][0]            
___________________________________________________________________________________________

In [30]:
vgg.model.optimizer.lr.set_value(0.00001)

In [31]:
# re-train the last layer of the model & save the weights accordingly
latest_weights_fn = None
num_arch += 1
for i in range(num_epochs):
    print "Running epoch: %d" % i
    vgg.model.fit_generator(batches,samples_per_epoch=batches.nb_sample,nb_epoch=1,
                            validation_data=val_batches,nb_val_samples=val_batches.nb_sample)
#    vgg.fit(batches,val_batches)
    latest_weights_fn = 'ft%d_%d.h5' % (num_arch, i)
    vgg.model.save_weights(results_path+latest_weights_fn)
print "Completed %d fit operations" % num_epochs

Running epoch: 0
Epoch 1/1
20424/20424 [==============================] - 551s - loss: 0.7386 - acc: 0.7660 - val_loss: 0.2264 - val_acc: 0.9440
Completed 1 fit operations


### From Scratch - 1

In [27]:
# Build Keras model architecture from scratch
model = Sequential()

# Preprocessing
model.add(BatchNormalization(axis=1,input_shape=(3,224,224)))

# Convolution
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(nb_filter=16,nb_row=3,nb_col=3,activation='relu'))
model.add(BatchNormalization(axis=1))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(nb_filter=16,nb_row=3,nb_col=3,activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
model.add(BatchNormalization(axis=1))

# Fully connected
model.add(Flatten())
model.add(Dense(1028,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Dense(10,activation='softmax'))

In [28]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
batchnormalization_45 (BatchNorma(None, 3, 224, 224)   6           batchnormalization_input_5[0][0] 
____________________________________________________________________________________________________
zeropadding2d_23 (ZeroPadding2D) (None, 3, 226, 226)   0           batchnormalization_45[0][0]      
____________________________________________________________________________________________________
convolution2d_23 (Convolution2D) (None, 16, 224, 224)  448         zeropadding2d_23[0][0]           
____________________________________________________________________________________________________
batchnormalization_46 (BatchNorma(None, 16, 224, 224)  32          convolution2d_23[0][0]           
___________________________________________________________________________________________

In [29]:
# Compile
model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [30]:
# Train
gen = image.ImageDataGenerator()
batch_size = 64
num_epochs = 3
#arch = 0

In [31]:
batches = gen.flow_from_directory(directory=train_path,
                                  batch_size=batch_size,target_size=(224,224))
val_batches = gen.flow_from_directory(directory=valid_path,
                                      batch_size=2*batch_size,target_size=(224,224))

Found 20424 images belonging to 10 classes.
Found 2000 images belonging to 10 classes.


In [18]:
imgs, labels = next(batches)

plots(imgs,titles=labels)

In [32]:
model.fit_generator(batches,samples_per_epoch=batches.nb_sample,nb_epoch=num_epochs,
                    validation_data=val_batches,nb_val_samples=val_batches.nb_sample)
latest_weights = 'scratch_model%d.h5' %arch
model.save_weights(results_path+latest_weights)
arch += 1

Epoch 1/3
20424/20424 [==============================] - 287s - loss: 0.4144 - acc: 0.9108 - val_loss: 0.4388 - val_acc: 0.8640
Epoch 2/3
20424/20424 [==============================] - 285s - loss: 0.0113 - acc: 0.9981 - val_loss: 0.0114 - val_acc: 0.9980
Epoch 3/3
20424/20424 [==============================] - 280s - loss: 0.0036 - acc: 0.9995 - val_loss: 0.0049 - val_acc: 0.9985


### From Scratch - 2

In [196]:
# Build Keras model architecture from scratch
model = Sequential()

# Preprocessing
model.add(BatchNormalization(axis=1,input_shape=(3,224,224)))

# Convolution
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(nb_filter=64,nb_row=3,nb_col=3,activation='relu'))
model.add(BatchNormalization(axis=1))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(nb_filter=64,nb_row=3,nb_col=3,activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
model.add(BatchNormalization(axis=1))

model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(nb_filter=64,nb_row=3,nb_col=3,activation='relu'))
model.add(BatchNormalization(axis=1))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(nb_filter=64,nb_row=3,nb_col=3,activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
model.add(BatchNormalization(axis=1))

model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(nb_filter=64,nb_row=3,nb_col=3,activation='relu'))
model.add(BatchNormalization(axis=1))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(nb_filter=64,nb_row=3,nb_col=3,activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
model.add(BatchNormalization(axis=1))

model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(nb_filter=64,nb_row=3,nb_col=3,activation='relu'))
model.add(BatchNormalization(axis=1))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(nb_filter=64,nb_row=3,nb_col=3,activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
model.add(BatchNormalization(axis=1))

# Fully connected
model.add(Flatten())
p = 0.
model.add(Dense(4096,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(p))
model.add(Dense(2048,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(p))
model.add(Dense(1024,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(p))
model.add(Dense(1024,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(p))
model.add(Dense(1024,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(p))
model.add(Dense(128,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(p))
model.add(Dense(128,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(p))
model.add(Dense(128,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(p))
model.add(Dense(10,activation='softmax'))

In [197]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
batchnormalization_256 (BatchNorm(None, 3, 224, 224)   6           batchnormalization_input_20[0][0]
____________________________________________________________________________________________________
zeropadding2d_173 (ZeroPadding2D)(None, 3, 226, 226)   0           batchnormalization_256[0][0]     
____________________________________________________________________________________________________
convolution2d_173 (Convolution2D)(None, 64, 224, 224)  1792        zeropadding2d_173[0][0]          
____________________________________________________________________________________________________
batchnormalization_257 (BatchNorm(None, 64, 224, 224)  128         convolution2d_173[0][0]          
___________________________________________________________________________________________

In [198]:
# Compile
model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [212]:
# Train
batch_size = 32
num_epochs = 3
#arch = 5

In [213]:
gen = image.ImageDataGenerator(rotation_range=12, width_shift_range=0.15, 
       height_shift_range=0.15, shear_range=0.15, zoom_range=0.2, 
       channel_shift_range=10., dim_ordering='th')
batches = gen.flow_from_directory(directory=train_path,
                                  batch_size=batch_size,target_size=(224,224))
val_batches = gen.flow_from_directory(directory=valid_path,
                                      batch_size=2*batch_size,target_size=(224,224))

Found 19455 images belonging to 10 classes.
Found 2969 images belonging to 10 classes.


In [201]:
#imgs, labels = next(batches)

#plots(imgs[:4],titles=labels[:4])

In [214]:
model.fit_generator(batches,samples_per_epoch=batches.nb_sample,nb_epoch=num_epochs,
                    validation_data=val_batches,nb_val_samples=val_batches.nb_sample)
latest_weights = 'scratch_model%d.h5' %arch
model.save_weights(results_path+latest_weights)
arch += 1

Epoch 1/3
19455/19455 [==============================] - 923s - loss: 0.1434 - acc: 0.9562 - val_loss: 0.3670 - val_acc: 0.8868
Epoch 2/3
19455/19455 [==============================] - 922s - loss: 0.1340 - acc: 0.9600 - val_loss: 0.3159 - val_acc: 0.8993
Epoch 3/3
19455/19455 [==============================] - 923s - loss: 0.1130 - acc: 0.9667 - val_loss: 0.2870 - val_acc: 0.9017


### From Scratch - 3

In [32]:
# Build Keras model architecture from scratch
model = Sequential()

# Preprocessing
model.add(BatchNormalization(axis=1,input_shape=(3,224,224)))

# Convolution
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(nb_filter=64,nb_row=3,nb_col=3,activation='relu'))
model.add(BatchNormalization(axis=1))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(nb_filter=64,nb_row=3,nb_col=3,activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
model.add(BatchNormalization(axis=1))

model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(nb_filter=128,nb_row=3,nb_col=3,activation='relu'))
model.add(BatchNormalization(axis=1))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(nb_filter=128,nb_row=3,nb_col=3,activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
model.add(BatchNormalization(axis=1))

model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(nb_filter=256,nb_row=3,nb_col=3,activation='relu'))
model.add(BatchNormalization(axis=1))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(nb_filter=256,nb_row=3,nb_col=3,activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
model.add(BatchNormalization(axis=1))

model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(nb_filter=512,nb_row=3,nb_col=3,activation='relu'))
model.add(BatchNormalization(axis=1))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(nb_filter=512,nb_row=3,nb_col=3,activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
model.add(BatchNormalization(axis=1))

model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(nb_filter=512,nb_row=3,nb_col=3,activation='relu'))
model.add(BatchNormalization(axis=1))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(nb_filter=512,nb_row=3,nb_col=3,activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
model.add(BatchNormalization(axis=1))

# Fully connected
model.add(Flatten())
p = 0.6
model.add(Dense(4096,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(p))
model.add(Dense(4096,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(p))
model.add(Dense(10,activation='softmax'))

In [33]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
batchnormalization_65 (BatchNorma(None, 3, 224, 224)   6           batchnormalization_input_5[0][0] 
____________________________________________________________________________________________________
zeropadding2d_41 (ZeroPadding2D) (None, 3, 226, 226)   0           batchnormalization_65[0][0]      
____________________________________________________________________________________________________
convolution2d_41 (Convolution2D) (None, 64, 224, 224)  1792        zeropadding2d_41[0][0]           
____________________________________________________________________________________________________
batchnormalization_66 (BatchNorma(None, 64, 224, 224)  128         convolution2d_41[0][0]           
___________________________________________________________________________________________

In [34]:
# Compile
model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [35]:
# Train
batch_size = 64
num_epochs = 1
#arch = 23

In [36]:
gen = image.ImageDataGenerator(rotation_range=12, width_shift_range=0.15, 
       height_shift_range=0.15, shear_range=0.15, zoom_range=0.2, 
       channel_shift_range=10., dim_ordering='th')
batches = gen.flow_from_directory(directory=train_path,
                                  batch_size=batch_size,target_size=(224,224))
val_batches = gen.flow_from_directory(directory=valid_path,
                                      batch_size=2*batch_size,target_size=(224,224))

Found 19455 images belonging to 10 classes.
Found 2969 images belonging to 10 classes.


In [37]:
#imgs, labels = next(batches)

#plots(imgs[:4],titles=labels[:4])

In [39]:
model.fit_generator(batches,samples_per_epoch=batches.nb_sample,nb_epoch=num_epochs,
                    validation_data=val_batches,nb_val_samples=val_batches.nb_sample)
latest_weights = 'scratch_model%d.h5' %arch
model.save_weights(results_path+latest_weights)
arch += 1

Epoch 1/3
19455/19455 [==============================] - 875s - loss: 0.6976 - acc: 0.7630 - val_loss: 2.2901 - val_acc: 0.4746
Epoch 2/3
19455/19455 [==============================] - 876s - loss: 0.5552 - acc: 0.8180 - val_loss: 0.7413 - val_acc: 0.7767
Epoch 3/3
19455/19455 [==============================] - 875s - loss: 0.4285 - acc: 0.8641 - val_loss: 0.8372 - val_acc: 0.7413


## Generate & valid predictions

In [215]:
# generate predictions
gen = image.ImageDataGenerator()
test_batches = gen.flow_from_directory(test_path,target_size=(224,224),
                                       shuffle=False,class_mode=None,batch_size=batch_size)

Found 79726 images belonging to 1 classes.


In [216]:
preds = model.predict_generator(test_batches,val_samples=test_batches.nb_sample)

In [217]:
print preds[:5]

filenames = test_batches.filenames
print filenames[:5]

[[  5.0003e-03   5.5157e-02   1.0565e-02   2.4860e-04   7.7472e-04   4.6769e-04   2.7351e-01
    7.5514e-03   1.2908e-01   5.1765e-01]
 [  7.7406e-01   1.6244e-03   2.2802e-04   2.0790e-04   5.5991e-05   1.1218e-02   4.0555e-05
    1.2217e-01   1.0642e-02   7.9751e-02]
 [  2.7501e-04   9.6837e-05   8.9538e-04   1.5659e-04   9.9592e-01   8.9740e-05   1.7367e-04
    1.5768e-05   2.1299e-03   2.5108e-04]
 [  4.2851e-04   1.1455e-06   7.5924e-04   7.6887e-06   8.7917e-05   2.8458e-05   7.9572e-04
    3.0907e-04   9.9682e-01   7.6335e-04]
 [  1.0056e-02   9.7209e-01   1.6908e-03   3.4534e-03   1.1346e-03   1.6580e-04   7.7855e-03
    5.2233e-04   1.5933e-03   1.5079e-03]]
['unknown/img_81601.jpg', 'unknown/img_14887.jpg', 'unknown/img_62885.jpg', 'unknown/img_45125.jpg', 'unknown/img_22633.jpg']


In [218]:
# save predictions
save_array(results_path+'test_preds.dat',preds)
save_array(results_path+'filenames.dat',filenames)

In [219]:
# validate predictions
# 

## Submit

In [220]:
# extra the img filenames
img_filenames = np.array([f[8:] for f in filenames])

In [221]:
# clean / clip probabilities
def do_clip(array, mx): return np.clip(array, (1-mx)/9,mx)

In [222]:
gen = image.ImageDataGenerator()
val_batches = gen.flow_from_directory(directory=valid_path,shuffle=False,
                                      batch_size=1,target_size=(224,224))
val_classes = val_batches.classes
val_labels = onehot(val_classes)
val_preds = model.predict_generator(val_batches,val_samples=val_batches.nb_sample)

Found 2969 images belonging to 10 classes.


In [228]:
keras.metrics.categorical_crossentropy(val_labels, do_clip(val_preds, 0.94)).eval()

array(0.16744581952881796)

In [250]:
# test different maxes
preds_clipped = do_clip(preds, 0.97)

In [251]:
# join the arrays
subm2 = np.zeros(img_filenames.size,dtype=[('img',img_filenames.dtype.str),('probs',preds.dtype.str,(10,))])
subm2['img'] = img_filenames
subm2['probs'] = preds_clipped
subm2[:5]

array([ ('img_81601.jpg', [0.0050003016367554665, 0.05515734851360321, 0.010564645752310753, 0.0033333334140479565, 0.0033333334140479565, 0.0033333334140479565, 0.2735058665275574, 0.00755135715007782, 0.12907674908638, 0.517652690410614]),
       ('img_14887.jpg', [0.7740589380264282, 0.0033333334140479565, 0.0033333334140479565, 0.0033333334140479565, 0.0033333334140479565, 0.01121825072914362, 0.0033333334140479565, 0.12217257916927338, 0.010642309673130512, 0.07975097745656967]),
       ('img_62885.jpg', [0.0033333334140479565, 0.0033333334140479565, 0.0033333334140479565, 0.0033333334140479565, 0.9700000286102295, 0.0033333334140479565, 0.0033333334140479565, 0.0033333334140479565, 0.0033333334140479565, 0.0033333334140479565]),
       ('img_45125.jpg', [0.0033333334140479565, 0.0033333334140479565, 0.0033333334140479565, 0.0033333334140479565, 0.0033333334140479565, 0.0033333334140479565, 0.0033333334140479565, 0.0033333334140479565, 0.9700000286102295, 0.0033333334140479565]),


In [247]:
stall = np.dtype([('img',img_filenames.dtype.str)] + preds.dtype.descr*preds.shape[1])
subm = subm2.view(stall)
fmt = '%s' + preds.shape[1] * ',%.4f'

In [252]:
num_subm = 12

In [253]:
%cd $DATA_HOME_DIR
num_subm += 1
submission_filename = 'submission%d.csv' % num_subm
np.savetxt(submission_filename,subm,fmt,
           header='img,c0,c1,c2,c3,c4,c5,c6,c7,c8,c9',delimiter=',',comments='')
!head $DATA_HOME_DIR/$submission_filename

/home/ubuntu/nbs/data/statefarm
img,c0,c1,c2,c3,c4,c5,c6,c7,c8,c9
img_81601.jpg,0.0067,0.0552,0.0106,0.0067,0.0067,0.0067,0.2735,0.0076,0.1291,0.5177
img_14887.jpg,0.7741,0.0067,0.0067,0.0067,0.0067,0.0112,0.0067,0.1222,0.0106,0.0798
img_62885.jpg,0.0067,0.0067,0.0067,0.0067,0.9400,0.0067,0.0067,0.0067,0.0067,0.0067
img_45125.jpg,0.0067,0.0067,0.0067,0.0067,0.0067,0.0067,0.0067,0.0067,0.9400,0.0067
img_22633.jpg,0.0101,0.9400,0.0067,0.0067,0.0067,0.0067,0.0078,0.0067,0.0067,0.0067
img_45660.jpg,0.0067,0.0067,0.0067,0.0067,0.0067,0.0067,0.0068,0.9105,0.0728,0.0067
img_88962.jpg,0.0067,0.0067,0.0067,0.0067,0.0067,0.0067,0.0067,0.9400,0.0067,0.0067
img_92987.jpg,0.0067,0.0067,0.9400,0.0067,0.0067,0.0067,0.0067,0.0067,0.0072,0.0067
img_26932.jpg,0.7451,0.0067,0.0067,0.0067,0.0067,0.0125,0.0067,0.0067,0.0179,0.2142


In [254]:
%cd $DATA_HOME_DIR

!kg submit $submission_filename -c state-farm-distracted-driver-detection

/home/ubuntu/nbs/data/statefarm
Starting new HTTPS connection (1): www.kaggle.com

